[Data](https://www.microns-explorer.org/publications) from [Microns](https://www.microns-explorer.org/acknowledgments) Layer 2/3 electron microscopy volume   
Mesh and synapse code modified from Forrest Collman, [Allen Institute](https://github.com/AllenInstitute/MicronsBinder/blob/master/notebooks/intro/MeshExample.ipynb)

In [1]:
from dash import Dash, html, dcc, Output, Input
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template
import pandas as pd
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

In [2]:
# run this cell to download meshes from 65 "clean and complete" curated pyr neurons
# and 34 inhibitory neurons in the Layer 2/3 volume
# this will put the large mesh files on your local machine in the mesh_dir defined above
# this cell only needs to be run once and will take 10 or more minutes to complete

# define directory location to mesh files
# mesh_dir = 'data/neuron_meshes_v185/' # or change to your desired folder
# seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
# mm = trimesh_io.MeshMeta(cv_path=seg_source,
#                          disk_cache_path=mesh_dir, 
#                          cache_size=20)

#clean_complete = pd.read_csv("data/clean_and_complete_soma_ids_v185.csv", names=['cellid'])
#clean_complete_list = clean_complete.cellid.to_list()
#for i in range(len(clean_complete_list)):
#    downloadmesh = mm.mesh(seg_id = clean_complete_list[i], remove_duplicate_vertices=True)

#inhib = pd.read_csv("data/layer23_inhibitory_neurons_cellids.csv",  index_col=[0])
#inhib.rename(columns={'cell_segid':'cellid'}, inplace=True)
#inhib_list = inhib.cellid.to_list()
#for i in range(len(inhib_list)):
#    downloadmesh = mm.mesh(seg_id = inhib_list[i], remove_duplicate_vertices=True)


In [3]:
# read in data tables
clean_complete = pd.read_csv("data/clean_and_complete_soma_ids_v185.csv", names=['cellid'], dtype={'cellid':str})
syn_clean_complete = pd.read_csv('data/syn_clean_complete.csv', index_col=[0])
inhib = pd.read_csv("data/layer23_inhibitory_neurons_cellids.csv", dtype={'cell_segid':str}, index_col=[0])
inhib.rename(columns={'cell_segid':'cellid'}, inplace=True)
syn_inhib = pd.read_csv('data/syn_inhibitory.csv', index_col=[0])

# define directory location to mesh files
mesh_dir = 'data/neuron_meshes_v185/' # or change to your desired folder
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

In [4]:
# setting up and loading pyr neuron mesh and synapse data into dictionaries
# this cell will take a minute or more to complete

## create a dictionary of meshes from 65 clean and complete pyr neurons
pyr_mesh_dictionary = {}
for i in range(len(clean_complete.cellid.values)):
    mesh_file = os.path.join(mesh_dir, str(clean_complete.cellid.values[i]) +'.h5')
    pyr_mesh_dictionary['cellid_' + str(clean_complete.cellid.values[i])] = mm.mesh(filename = mesh_file)

## create dictionaries for pre and post synaptic sites from 65 clean and complete pyr neurons
pyr_cell_post_subgraph_dict = {}
pyr_cell_pre_subgraph_dict = {}
for i in range(len(clean_complete.cellid.values)):
    pyr_cell_post_subgraph_dict['pyr_cell_post_subgraph_' + str(clean_complete.cellid.values[i])] = syn_clean_complete.query(f'post_root_id=={clean_complete.cellid.values[i]}')
    pyr_cell_pre_subgraph_dict['pyr_cell_pre_subgraph_' + str(clean_complete.cellid.values[i])] = syn_clean_complete.query(f'pre_root_id=={clean_complete.cellid.values[i]}')

## create dictionaries for the synapse coordinates from 65 clean and complete pyr neurons
pyr_postsyn_xyz_dict = {}
pyr_presyn_xyz_dict = {}
for i in range(len(clean_complete.cellid.values)):
    pyr_postsyn_xyz_dict['pyr_postsyn_xyz_dict_' + str(clean_complete.cellid.values[i])] = pyr_cell_post_subgraph_dict.get('pyr_cell_post_subgraph_' + str(clean_complete.cellid.values[i]))[['ctr_pt_x_nm', 'ctr_pt_y_nm', 'ctr_pt_z_nm']].values
    pyr_presyn_xyz_dict['pyr_presyn_xyz_dict_' + str(clean_complete.cellid.values[i])] = pyr_cell_pre_subgraph_dict.get('pyr_cell_pre_subgraph_' + str(clean_complete.cellid.values[i]))[['ctr_pt_x_nm', 'ctr_pt_y_nm', 'ctr_pt_z_nm']].values


In [5]:
# setting up and loading inhib neuron mesh and synapse data into dictionaries

## create a dictionary of meshes from the 34 inhibitor neurons
inh_mesh_dictionary = {}
for i in range(len(inhib.cellid.values)):
    mesh_file = os.path.join(mesh_dir, str(inhib.cellid.values[i]) +'.h5')
    inh_mesh_dictionary['cellid_' + str(inhib.cellid.values[i])] = mm.mesh(filename = mesh_file)

## create dictionaries for pre and post synaptic sites from 34 inhibitory neurons
inh_cell_post_subgraph_dict = {}
inh_cell_pre_subgraph_dict = {}
for i in range(len(inhib.cellid.values)):
    inh_cell_post_subgraph_dict['inh_cell_post_subgraph_' + str(inhib.cellid.values[i])] = syn_inhib.query(f'post_root_id=={inhib.cellid.values[i]}')
    inh_cell_pre_subgraph_dict['inh_cell_pre_subgraph_' + str(inhib.cellid.values[i])] = syn_inhib.query(f'pre_root_id=={inhib.cellid.values[i]}')

## create dictionaries for the synapse coordinates from 34 inhibitory neurons
inh_postsyn_xyz_dict = {}
inh_presyn_xyz_dict = {}
for i in range(len(inhib.cellid.values)):
    inh_postsyn_xyz_dict['inh_postsyn_xyz_dict_' + str(inhib.cellid.values[i])] = inh_cell_post_subgraph_dict.get('inh_cell_post_subgraph_' + str(inhib.cellid.values[i]))[['ctr_pt_x_nm', 'ctr_pt_y_nm', 'ctr_pt_z_nm']].values
    inh_presyn_xyz_dict['inh_presyn_xyz_dict_' + str(inhib.cellid.values[i])] = inh_cell_pre_subgraph_dict.get('inh_cell_pre_subgraph_' + str(inhib.cellid.values[i]))[['ctr_pt_x_nm', 'ctr_pt_y_nm', 'ctr_pt_z_nm']].values


In [6]:
# components

app = Dash(__name__, external_stylesheets=[dbc.themes.CERULEAN])
load_figure_template("CERULEAN")

pyr_title = html.H4(children='')
pyr_plot = dcc.Graph(figure={})
pyr_dropdown = dcc.Dropdown(options=clean_complete.cellid.values,
                        value=str(clean_complete.cellid.values[0]),
                        clearable=False,
                        style={"textAlign":"center"},
                        )
inh_title = html.H4(children='')
inh_plot = dcc.Graph(figure={})
inh_dropdown = dcc.Dropdown(options=inhib.cellid.values,
                        value=str(inhib.cellid.values[0]),
                        clearable=False,
                        style={"textAlign":"center"},
                        )
pyr_granularity = dcc.Slider(5, 50, 5, value=50, id='pyr_granularity')
inh_granularity = dcc.Slider(5, 50, 5, value=50, id='inh_granularity')

# layout

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#119DFF', 
    'color': 'white',
}

app.layout = html.Div(
    style={"textAlign":"center"},
    children = [
    dcc.Tabs(id="tabs",
             value='pyr_tab',
             children=[
                 dcc.Tab(
                     label='65 Clean and Complete Pyramidal Neurons', 
                     value='pyr_tab',
                     selected_style=tab_selected_style,
                     children=[
                         dbc.Row([
                             dbc.Col(
                                 width=3),
                             dbc.Col(
                                 [pyr_title])]),
                         dbc.Row([
                             dbc.Col([
                                 dcc.Markdown('Select granularity'), 
                                 pyr_granularity,
                                 dcc.Markdown('Select cellid'), 
                                 pyr_dropdown], width=3),
                             dbc.Col(dbc.Card([pyr_plot]), width=9),
                         ]),
                     ]),
                 dcc.Tab(
                     label='34 Inhibitory neurons', 
                     value='inh_tab',
                     selected_style=tab_selected_style,
                     children=[
                         dbc.Row([
                             dbc.Col(
                                 width=3),
                             dbc.Col(
                                 [inh_title])]),
                         dbc.Row([
                             dbc.Col([
                                 dcc.Markdown('Select granularity'), 
                                 inh_granularity,
                                 dcc.Markdown('Select cellid'), 
                                 inh_dropdown], width=3),
                             dbc.Col(dbc.Card([inh_plot]), width=9),
                         ]),
                     ]),
                 
             ]
            , className='dbc')
    ]
)

# callback

@app.callback(
    Output(pyr_plot, 'figure'),
    Output(pyr_title, 'children'),
    Output(inh_plot, 'figure'),
    Output(inh_title, 'children'),
    Input(pyr_dropdown, 'value'),
    Input(inh_dropdown, 'value'),
    Input(pyr_granularity, 'value'),
    Input(inh_granularity, 'value')
)

def update_graph(pyr_cellid, inh_cellid, pyr_granularity, inh_granularity):

    pyr_fig = go.Figure(data=[go.Scatter3d(
        x=pyr_mesh_dictionary.get('cellid_' + str(pyr_cellid)).vertices[::pyr_granularity,0],
        y=pyr_mesh_dictionary.get('cellid_' + str(pyr_cellid)).vertices[::pyr_granularity,1],
        z=pyr_mesh_dictionary.get('cellid_' + str(pyr_cellid)).vertices[::pyr_granularity,2],
        mode='markers',
        marker=dict(size=1),
        showlegend=False),
        ])
    
    camera = dict(eye=dict(x=0, y=0.1, z=1.5))
    pyr_fig.update_layout(scene_camera=camera, scene_dragmode='orbit')
    
    pyr_fig.add_trace(
        go.Scatter3d(
            x=pyr_postsyn_xyz_dict.get('pyr_postsyn_xyz_dict_' + str(pyr_cellid))[:,0],
            y=pyr_postsyn_xyz_dict.get('pyr_postsyn_xyz_dict_' + str(pyr_cellid))[:,1],
            z=pyr_postsyn_xyz_dict.get('pyr_postsyn_xyz_dict_' + str(pyr_cellid))[:,2],
            mode="markers",
            marker=dict(size=2, opacity=0.95),
            showlegend=False)
    )

    pyr_fig.add_trace(
        go.Scatter3d(
            x=pyr_presyn_xyz_dict.get('pyr_presyn_xyz_dict_' + str(pyr_cellid))[:,0],
            y=pyr_presyn_xyz_dict.get('pyr_presyn_xyz_dict_' + str(pyr_cellid))[:,1],
            z=pyr_presyn_xyz_dict.get('pyr_presyn_xyz_dict_' + str(pyr_cellid))[:,2],
            mode="markers",
            marker=dict(size=3, opacity=1),
            showlegend=False)
    )

    inh_fig = go.Figure(data=[go.Scatter3d(
        x=inh_mesh_dictionary.get('cellid_' + str(inh_cellid)).vertices[::inh_granularity,0],
        y=inh_mesh_dictionary.get('cellid_' + str(inh_cellid)).vertices[::inh_granularity,1],
        z=inh_mesh_dictionary.get('cellid_' + str(inh_cellid)).vertices[::inh_granularity,2],
        mode='markers',
        marker=dict(size=1),
        showlegend=False),
        ])
    
    camera = dict(eye=dict(x=0, y=0.1, z=1.5))
    inh_fig.update_layout(scene_camera=camera, scene_dragmode='orbit')

    inh_fig.add_trace(
        go.Scatter3d(
            x=inh_postsyn_xyz_dict.get('inh_postsyn_xyz_dict_' + str(inh_cellid))[:,0],
            y=inh_postsyn_xyz_dict.get('inh_postsyn_xyz_dict_' + str(inh_cellid))[:,1],
            z=inh_postsyn_xyz_dict.get('inh_postsyn_xyz_dict_' + str(inh_cellid))[:,2],
            mode="markers",
            marker=dict(size=2, opacity=0.95),
            showlegend=False)
    )

    inh_fig.add_trace(
        go.Scatter3d(
            x=inh_presyn_xyz_dict.get('inh_presyn_xyz_dict_' + str(inh_cellid))[:,0],
            y=inh_presyn_xyz_dict.get('inh_presyn_xyz_dict_' + str(inh_cellid))[:,1],
            z=inh_presyn_xyz_dict.get('inh_presyn_xyz_dict_' + str(inh_cellid))[:,2],
            mode="markers",
            marker=dict(size=3, opacity=1),
            showlegend=False)
    )

    return pyr_fig, 'cellid: '+pyr_cellid, inh_fig, 'cellid:'+inh_cellid

# run app
if __name__=='__main__':
    app.run(jupyter_mode="tab")

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>